# Post-pandemic financial analysis

In [1]:
# Import warnings + watermark
from watermark import watermark
from warnings import filterwarnings
filterwarnings("ignore")
print(watermark())
print(watermark(iversions=True, globals_=globals()))

Last updated: 2022-11-19T21:30:03.155761-08:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 22.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit




In [2]:
#import the necessary modules
import pandas as pd
import numpy as np
#questionary
from pathlib import Path
import fire
import questionary
#API
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline

In [3]:
POLYGON_API_KEY ='enZp2AUpH4pGXJJuQ1CbjdVXJIsBFBEl'

In [4]:
ticker_list = ["AMZN", "RTH", "AMT", "IYR", "XOM", "XLE", "SPY"]

In [5]:
import requests
import datetime
from time import sleep
from tqdm import tqdm
def get_prices(start_date,end_date,universe):
    data = pd.DataFrame()
    # end_at = datetime.datetime.now() 
    # begin_from = end_at + datetime.timedelta(days=-5000)
    for t in tqdm(universe):
        print(t)
        print(len(data))
        # for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
        #     end = start + datetime.timedelta(days = 88)
        #     end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
        #     start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
        try:
            r = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}')
        except json.JSONDecodeError:
            pass
        try:
            # print(r.json())
            temp_data = pd.DataFrame(r.json()['results'])
            temp_data['ticker'] = t
            data = data.append(temp_data, ignore_index=True)
            print(t+':'+str(pd.to_datetime(start_date, unit='s'))+':'+str(pd.to_datetime(end_date, unit='s')))
        except KeyError:
            print(f'{t} was not found')
            pass
        except ValueError:
            pass
        except ConnectionError as error:
            print(error)
            sleep(200)
            continue
        except TimeoutError as error:
            print(error)
            sleep(200)
            continue
        sleep(0.25)
#     data.to_csv(file_name,index = False)
    data.t = pd.to_datetime(data.t, unit = 'ms')
    columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
    data= data.rename(columns = columns_name)
    data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
    data = data.set_index(['time', 'ticker'])
    data["daily_returns"] = data['Close'].groupby('ticker').pct_change()
    data = data.dropna()
    #data.to_csv(file_name, index = False)
    return data

In [6]:
# Create `start_date` and `end_date` variables for the period before the pandemic
start_date = "2022-03-01"
end_date = "2022-11-01"
all_data = get_prices(start_date=start_date, end_date=end_date, universe=ticker_list)


  0%|                                                                                                                | 0/7 [00:00<?, ?it/s]

AMZN
0


 14%|██████████████▊                                                                                         | 1/7 [00:00<00:03,  1.81it/s]

RTH
171


 29%|█████████████████████████████▋                                                                          | 2/7 [00:01<00:03,  1.62it/s]

...

 71%|██████████████████████████████████████████████████████████████████████████▎                             | 5/7 [00:02<00:01,  1.74it/s]

XLE
855


 86%|█████████████████████████████████████████████████████████████████████████████████████████▏              | 6/7 [00:03<00:00,  1.75it/s]

SPY
1026


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.72it/s]


In [7]:
all_data


,,Open,Close,High,Low,Volume,daily_returns
time,ticker,,,,,,
2022-03-02 05:00:00,AMZN,150.8490,152.0525,152.9995,148.7500,47334720.0,0.006024
2022-03-03 05:00:00,AMZN,153.5315,147.8985,153.9290,146.8560,65197200.0,-0.027320
2022-03-04 05:00:00,AMZN,147.1590,145.6410,147.8500,143.8070,60983120.0,-0.015264
2022-03-07 05:00:00,AMZN,145.4435,137.4530,146.0945,137.4165,86933860.0,-0.056220
2022-03-08 05:00:00,AMZN,136.6835,136.0145,140.6995,133.5725,91661580.0,-0.010465
...,...,...,...,...,...,...,...
2022-10-26 04:00:00,SPY,381.6200,382.0200,387.5800,381.3500,103906848.0,-0.007534
2022-10-27 04:00:00,SPY,383.0700,379.9800,385.0000,379.3300,81971756.0,-0.005340
2022-10-28 04:00:00,SPY,379.8700,389.0200,389.5200,379.6800,100301958.0,0.023791


In [8]:
all_data.index

MultiIndex([('2022-03-02 05:00:00', 'AMZN'),
            ('2022-03-03 05:00:00', 'AMZN'),
            ('2022-03-04 05:00:00', 'AMZN'),
            ('2022-03-07 05:00:00', 'AMZN'),
            ('2022-03-08 05:00:00', 'AMZN'),
            ('2022-03-09 05:00:00', 'AMZN'),
            ('2022-03-10 05:00:00', 'AMZN'),
            ('2022-03-11 05:00:00', 'AMZN'),
            ('2022-03-14 04:00:00', 'AMZN'),
            ('2022-03-15 04:00:00', 'AMZN'),
            ...
            ('2022-10-19 04:00:00',  'SPY'),
            ('2022-10-20 04:00:00',  'SPY'),
            ('2022-10-21 04:00:00',  'SPY'),
            ('2022-10-24 04:00:00',  'SPY'),
            ('2022-10-25 04:00:00',  'SPY'),
            ('2022-10-26 04:00:00',  'SPY'),
            ('2022-10-27 04:00:00',  'SPY'),
            ('2022-10-28 04:00:00',  'SPY'),
            ('2022-10-31 04:00:00',  'SPY'),
            ('2022-11-01 04:00:00',  'SPY')],
           names=['time', 'ticker'], length=1190)

In [9]:
idx = pd.IndexSlice

In [10]:
AMZN_daily_returns_df = all_data.loc[idx[:,'AMZN'],['daily_returns']]
AMZN_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,AMZN,0.006024
2022-03-03 05:00:00,AMZN,-0.027320
2022-03-04 05:00:00,AMZN,-0.015264
2022-03-07 05:00:00,AMZN,-0.056220
2022-03-08 05:00:00,AMZN,-0.010465
...,...,...
2022-10-26 04:00:00,AMZN,-0.040962
2022-10-27 04:00:00,AMZN,-0.040636
2022-10-28 04:00:00,AMZN,-0.068043


In [11]:
RTH_daily_returns_df = all_data.loc[idx[:,'RTH'],['daily_returns']]
RTH_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,RTH,0.019711
2022-03-03 05:00:00,RTH,-0.004525
2022-03-04 05:00:00,RTH,-0.007071
2022-03-07 05:00:00,RTH,-0.027808
2022-03-08 05:00:00,RTH,-0.012732
...,...,...
2022-10-26 04:00:00,RTH,-0.001160
2022-10-27 04:00:00,RTH,-0.003589
2022-10-28 04:00:00,RTH,0.001099


In [12]:
AMT_daily_returns_df = all_data.loc[idx[:,'AMT'],['daily_returns']]
AMT_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,AMT,0.010720
2022-03-03 05:00:00,AMT,0.018355
2022-03-04 05:00:00,AMT,0.012923
2022-03-07 05:00:00,AMT,-0.026272
2022-03-08 05:00:00,AMT,-0.017455
...,...,...
2022-10-26 04:00:00,AMT,0.007109
2022-10-27 04:00:00,AMT,0.008938
2022-10-28 04:00:00,AMT,0.030350


In [13]:
IYR_daily_returns_df = all_data.loc[idx[:,'IYR'],['daily_returns']]
IYR_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,IYR,0.017253
2022-03-03 05:00:00,IYR,0.008238
2022-03-04 05:00:00,IYR,0.004902
2022-03-07 05:00:00,IYR,-0.020471
2022-03-08 05:00:00,IYR,-0.005566
...,...,...
2022-10-26 04:00:00,IYR,0.001952
2022-10-27 04:00:00,IYR,0.001827
2022-10-28 04:00:00,IYR,0.022247


In [14]:
XOM_daily_returns_df = all_data.loc[idx[:,'XOM'],['daily_returns']]
XOM_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,XOM,0.017178
2022-03-03 05:00:00,XOM,0.006333
2022-03-04 05:00:00,XOM,0.037636
2022-03-07 05:00:00,XOM,0.036033
2022-03-08 05:00:00,XOM,0.007576
...,...,...
2022-10-26 04:00:00,XOM,0.011900
2022-10-27 04:00:00,XOM,0.003827
2022-10-28 04:00:00,XOM,0.029289


In [15]:
XLE_daily_returns_df = all_data.loc[idx[:,'XLE'],['daily_returns']]
XLE_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,XLE,0.022864
2022-03-03 05:00:00,XLE,0.002743
2022-03-04 05:00:00,XLE,0.028857
2022-03-07 05:00:00,XLE,0.014755
2022-03-08 05:00:00,XLE,0.015719
...,...,...
2022-10-26 04:00:00,XLE,0.013611
2022-10-27 04:00:00,XLE,0.003611
2022-10-28 04:00:00,XLE,0.003485


In [16]:
SPY_daily_returns_df = all_data.loc[idx[:,'SPY'],['daily_returns']]
SPY_daily_returns_df

,,daily_returns
time,ticker,
2022-03-02 05:00:00,SPY,0.018396
2022-03-03 05:00:00,SPY,-0.004978
2022-03-04 05:00:00,SPY,-0.008125
2022-03-07 05:00:00,SPY,-0.029479
2022-03-08 05:00:00,SPY,-0.007582
...,...,...
2022-10-26 04:00:00,SPY,-0.007534
2022-10-27 04:00:00,SPY,-0.005340
2022-10-28 04:00:00,SPY,0.023791


In [17]:
tech_daily_returns = pd.concat([AMZN_daily_returns_df, RTH_daily_returns_df], keys=["AMZN","RTH"])
tech_daily_returns

daily_returns
     time                ticker               
AMZN 2022-03-02 05:00:00 AMZN         0.006024
     2022-03-03 05:00:00 AMZN        -0.027320
     2022-03-04 05:00:00 AMZN        -0.015264
     2022-03-07 05:00:00 AMZN        -0.056220
     2022-03-08 05:00:00 AMZN        -0.010465
...                                        ...
RTH  2022-10-26 04:00:00 RTH         -0.001160
     2022-10-27 04:00:00 RTH         -0.003589
     2022-10-28 04:00:00 RTH          0.001099
     2022-10-31 04:00:00 RTH         -0.006342
     2022-11-01 04:00:00 RTH         -0.012703

[340 rows x 1 columns]

In [18]:
RE_daily_returns = pd.concat([AMT_daily_returns_df, IYR_daily_returns_df], keys=["AMT","IYR"])
RE_daily_returns

daily_returns
    time                ticker               
AMT 2022-03-02 05:00:00 AMT          0.010720
    2022-03-03 05:00:00 AMT          0.018355
    2022-03-04 05:00:00 AMT          0.012923
    2022-03-07 05:00:00 AMT         -0.026272
    2022-03-08 05:00:00 AMT         -0.017455
...                                       ...
IYR 2022-10-26 04:00:00 IYR          0.001952
    2022-10-27 04:00:00 IYR          0.001827
    2022-10-28 04:00:00 IYR          0.022247
    2022-10-31 04:00:00 IYR         -0.001189
    2022-11-01 04:00:00 IYR         -0.001905

[340 rows x 1 columns]

In [19]:
energy_daily_returns = pd.concat([XOM_daily_returns_df, XLE_daily_returns_df], keys=["XOM","XLE"])
energy_daily_returns

daily_returns
    time                ticker               
XOM 2022-03-02 05:00:00 XOM          0.017178
    2022-03-03 05:00:00 XOM          0.006333
    2022-03-04 05:00:00 XOM          0.037636
    2022-03-07 05:00:00 XOM          0.036033
    2022-03-08 05:00:00 XOM          0.007576
...                                       ...
XLE 2022-10-26 04:00:00 XLE          0.013611
    2022-10-27 04:00:00 XLE          0.003611
    2022-10-28 04:00:00 XLE          0.003485
    2022-10-31 04:00:00 XLE          0.008403
    2022-11-01 04:00:00 XLE          0.009667

[340 rows x 1 columns]

### A) Analyzing Each Stock and ETF Individually


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
from pytz import timezone
#from IPython.display import clear_output
from time import sleep
import numpy as np
import pyfolio as pf
import sqlalchemy

In [22]:

pre_pandemic_prices_df = pd.concat([AMZN_daily_returns_df.reset_index(level = 1,drop=True), RTH_daily_returns_df.reset_index(level = 1,drop=True), AMT_daily_returns_df.reset_index(level = 1,drop=True), IYR_daily_returns_df.reset_index(level = 1,drop=True), XOM_daily_returns_df.reset_index(level = 1,drop=True), XLE_daily_returns_df.reset_index(level = 1,drop=True), SPY_daily_returns_df.reset_index(level = 1,drop=True)],axis=0, keys=["AMZN","RTH","AMT","IYR","XOM","XLE","SPY"])
pre_pandemic_prices_df



daily_returns
     time                              
AMZN 2022-03-02 05:00:00       0.006024
     2022-03-03 05:00:00      -0.027320
     2022-03-04 05:00:00      -0.015264
     2022-03-07 05:00:00      -0.056220
     2022-03-08 05:00:00      -0.010465
...                                 ...
SPY  2022-10-26 04:00:00      -0.007534
     2022-10-27 04:00:00      -0.005340
     2022-10-28 04:00:00       0.023791
     2022-10-31 04:00:00      -0.007223
     2022-11-01 04:00:00      -0.004376

[1190 rows x 1 columns]

In [23]:
database_connection_string = 'sqlite:///'

In [24]:
engine = sqlalchemy.create_engine(database_connection_string)

In [25]:
engine.table_names()

[]

In [26]:
#Summary statistics for AMZN stock using .std() and .mean()
display(AMZN_daily_returns_df.std())
display(AMZN_daily_returns_df.mean())

daily_returns    0.031738
dtype: float64

daily_returns   -0.002111
dtype: float64

In [27]:
#Summary statistics for RTH stock using .std() and .mean()
RTH_daily_returns_df.std()
RTH_daily_returns_df.mean()

daily_returns   -0.000367
dtype: float64

In [28]:
#Summary statistics for AMT stock using .std() and .mean()
AMT_daily_returns_df.std()
AMT_daily_returns_df.mean()

daily_returns   -0.000353
dtype: float64

In [29]:
#Summary statistics for IYR stock using .std() and .mean()
IYR_daily_returns_df.std()
IYR_daily_returns_df.mean()

daily_returns   -0.000995
dtype: float64

In [30]:
#Summary statistics for XOM stock using .std() and .mean()
XOM_daily_returns_df.std()
XOM_daily_returns_df.mean()

daily_returns    0.002322
dtype: float64

In [31]:
#Summary statistics for XLE stock using .std() and .mean()
XLE_daily_returns_df.std()
XLE_daily_returns_df.mean()

daily_returns    0.001711
dtype: float64

In [32]:
pre_pandemic_prices_df.describe()

,daily_returns
count,1190.000000
mean,-0.000046
std,0.021852
min,-0.140494
25%,-0.011691
50%,0.000459
75%,0.014347
max,0.103615


In [33]:
#Summary statistics for AMZN using .describe()
AMZN_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,-0.002111
std,0.031738
min,-0.140494
25%,-0.020766
50%,-0.000736
75%,0.021655
max,0.103615


In [34]:
#Summary statistics for RTH using .describe()
RTH_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,-0.000367
std,0.017069
min,-0.074149
25%,-0.010150
50%,-0.001131
75%,0.012610
max,0.044233


In [35]:
#Summary statistics for AMT using .describe()
AMT_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,-0.000353
std,0.020251
min,-0.053181
25%,-0.013277
50%,0.001093
75%,0.012928
max,0.054639


In [36]:
#Summary statistics for IYR using .describe()
IYR_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,-0.000995
std,0.015847
min,-0.049313
25%,-0.010363
50%,-0.001608
75%,0.010670
max,0.039843


In [37]:
#Summary statistics for XOM using .describe()
XOM_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,0.002322
std,0.023824
min,-0.078853
25%,-0.012521
50%,0.004781
75%,0.018913
max,0.062239


In [38]:
#Summary statistics for XLE using .describe()
XLE_daily_returns_df.describe()

,daily_returns
count,170.000000
mean,0.001711
std,0.023771
min,-0.082730
25%,-0.014380
50%,0.003835
75%,0.017471
max,0.056512


In [39]:
# Variance Of Amzn
AMZN_var=AMZN_daily_returns_df.var()
AMZN_var

daily_returns    0.001007
dtype: float64

In [40]:
# Variance For RTH
RTH_var=RTH_daily_returns_df.var()
RTH_var

daily_returns    0.000291
dtype: float64

In [41]:
# Variance Of IYR
IYR_var=IYR_daily_returns_df.var()
IYR_var

daily_returns    0.000251
dtype: float64

In [42]:
# Variance Of AMT
AMT_var=AMT_daily_returns_df.var()
AMT_var

daily_returns    0.00041
dtype: float64

In [43]:
# Variance Of XOM
XOM_var=XOM_daily_returns_df.var()
XOM_var

daily_returns    0.000568
dtype: float64

In [44]:
# Variance Of XLE
XLE_var=XLE_daily_returns_df.var()
XLE_var

daily_returns    0.000565
dtype: float64

In [45]:
# Covariance for Each Stock and ETF
AMZN_Cov=AMZN_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
AMZN_Cov

0.00040044993167342205

In [46]:
RTH_Cov=RTH_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
RTH_Cov

0.00024378453298000983

In [47]:
# Covariance for Each Stock and ETF
AMT_Cov=AMT_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
AMT_Cov

0.00019650976184925383

In [48]:
IYR_Cov=IYR_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
IYR_Cov

0.0002019051817760291

In [49]:
XOM_Cov=XOM_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
XOM_Cov

0.00014200594731934434

In [50]:
XLE_Cov=XLE_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
XLE_Cov

0.00017020855755096786

In [51]:
year_trading_days = 252

In [52]:
# Annualized returns of RTH
RTH_Annual_Ret = RTH_daily_returns_df.mean()*252
RTH_Annual_Ret

daily_returns   -0.092406
dtype: float64

In [53]:
# Annualized returns of IYR
IYR_Annual_Ret = IYR_daily_returns_df.mean()*252
IYR_Annual_Ret

daily_returns   -0.25075
dtype: float64

In [54]:
# Annualized returns of XLE
XLE_Annual_Ret = XLE_daily_returns_df.mean()*252
XLE_Annual_Ret

daily_returns    0.431295
dtype: float64

In [55]:
# Annualized std dev of RTH
annual_std_dev_RTH = RTH_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_RTH

daily_returns    0.270963
dtype: float64

In [56]:
# Annualized std dev of IYR
annual_std_dev_IYR = IYR_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_IYR

daily_returns    0.251571
dtype: float64

In [57]:
# Annualized std dev of XLE
annual_std_dev_XLE = XLE_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_XLE

daily_returns    0.377347
dtype: float64

In [58]:
result = pf.create_returns_tear_sheet(AMZN_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amzn_prepan_tearsheet.png", format = "png")

Start date,2022-03-02
End date,2022-11-01
Total months,8
,Backtest
Annual return,-48.3%
Cumulative returns,-36.0%
Annual volatility,50.4%
Sharpe ratio,-1.06
Calmar ratio,-1.13
Stability,0.26
Max drawdown,-42.8%


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
result = pf.create_returns_tear_sheet(RTH_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("RTH_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(AMT_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amt_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(IYR_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("iyr_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XOM_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xom_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XLE_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xle_prepan_tearsheet.png", format = "png")


In [ ]:
result = pf.create_returns_tear_sheet(SPY_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("spy_prepan_tearsheet.png", format = "png")